In [3]:
import pandas as pd
import folium
import math

# ---- CSV laden ----
df = pd.read_csv("Baum999.csv", sep=";", encoding="utf-8")

# ---- EPSG:3857 -> WGS84 Umrechnung ----
R = 6378137.0
def mercator_to_wgs84(x, y):
    lon = (x / R) * (180 / math.pi)
    lat = (2 * math.atan(math.exp(y / R)) - math.pi/2) * (180 / math.pi)
    return lon, lat

df["lon"], df["lat"] = zip(*df.apply(lambda row: mercator_to_wgs84(row["x"], row["y"]), axis=1))

# ---- Parametrische Styles: Farben anhand "Kategorie" ----
styles = {
    "rot":   {"color": "red",   "fillColor": "red",   "radius": 4},
    "gruen": {"color": "green", "fillColor": "green", "radius": 4},
    "grün":  {"color": "green", "fillColor": "green", "radius": 4},
    "blau":  {"color": "blue",  "fillColor": "blue",  "radius": 4},
}

# ---- Karte erstellen ----
m = folium.Map(location=[52.51, 13.4], zoom_start=11, control_scale=True)

# ---- Marker mit Tooltip einfügen ----
for _, row in df.iterrows():
    kat = row["Kategorie"].strip().lower()  # "rot", "gruen", "blau"
    if kat in styles:
        s = styles[kat]
    else:
        s = {"color": "gray", "fillColor": "gray", "radius": 6}  # fallback

    info = f"""
    <b>{row['Objektname']}</b><br>
    {row['Bezirk']}<br>
    {row['Art Botanisch']}
    """
    folium.CircleMarker(
        location=[row["lat"], row["lon"]],
        radius=s["radius"],
        color=s["color"],
        fill=True,
        fill_color=s["fillColor"],
        fill_opacity=1.0,
        tooltip=info
    ).add_to(m)

# Karte an alle Punkte anpassen
m.fit_bounds(m.get_bounds())

# Fester Name um nicht im Versionskontrollsystem zu spammen
m.get_root().html.add_child(folium.Element('<div id="mapid" style="width:100%; height:100%;"></div>'))
m.get_name = lambda: "mapid"

# ---- Karte speichern ----
m.save("baumkarte.html")
print("Karte gespeichert als 'baumkarte.html'")


Karte gespeichert als 'baumkarte.html'
